In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

pd.set_option("display.precision", 1)

## On récupère la donnée

In [120]:
test_customers = pd.read_csv('data/test_customers.csv')
test_locations = pd.read_csv('data/test_locations.csv')

train_customers = pd.read_csv('data/train_customers.csv')
train_locations = pd.read_csv('data/train_locations.csv')

orders = pd.read_csv('data/orders.csv')
vendors = pd.read_csv('data/vendors.csv')


/tmp/ipykernel_22711/3811830009.py:7: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('data/orders.csv')


In [121]:
print('train_customers shape: ', train_customers.shape)
print('test_customers shape: ', test_customers.shape)
train_customers.head()

train_customers shape:  (34674, 8)
test_customers shape:  (9768, 8)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [122]:
print('train_locations shape: ', train_locations.shape)
print('test_locations shape: ', test_locations.shape)
train_locations.head()

train_locations shape:  (59503, 5)
test_locations shape:  (16720, 5)


,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.7,-78.8
1,02SFNJH,1,NaN,1.7,0.8
2,02SFNJH,2,NaN,-0.5,0.7
3,RU43CXC,0,Home,0.1,0.4
4,BDFBPRD,0,NaN,2.5,0.7


In [123]:
print('orders shape: ', orders.shape)
orders.head()

orders shape:  (135303, 26)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [124]:
print('vendors shape: ', vendors.shape)
vendors.head()

vendors shape:  (100, 59)


,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.6,0.8,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.5,0.7,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.4,0.6,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.6,0.8,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.5,0.6,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


### On arrange et on nettoie la donnée pour notre réseau

In [125]:
# merge train_customers(akeed_customer_id) and train_locations(customer_id)
train_customers_locations = pd.merge(train_customers, train_locations, left_on='akeed_customer_id', right_on='customer_id', how='inner')
print('train_customers_locations shape: ', train_customers_locations.shape)
train_customers_locations.head()

train_customers_locations shape:  (58213, 13)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,0,Work,-96.4,-67.2
1,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,1,Home,-96.6,-67.2
2,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,2,NaN,-0.1,-78.6
3,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,0,Home,-0.2,-78.6
4,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,1,Home,0.2,-78.6


In [126]:
# merge orders(vendor_id) and vendors(id)
orders_vendors = pd.merge(orders, vendors, left_on='vendor_id', right_on='id', how='inner')
print('train_customers_locations_orders_vendors shape: ', orders_vendors.shape)
orders_vendors.head()

train_customers_locations_orders_vendors shape:  (135303, 85)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at_y,updated_at,device_type,display_orders
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
1,163247.0,I9DNSMJ,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
2,163248.0,I9DNSMJ,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
3,163579.0,4BQI76G,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1
4,163609.0,V6JC4VF,4.0,20.4,1,NaN,0.0,NaN,NaN,No,...,1.0,"1,5,91,29,27","American,Burgers,Free Delivery,Hot Dogs,Pasta",Y,1.0,1.0,2018-10-19 12:38:55,2020-04-07 23:41:41,3,1


In [127]:
# same for test data
test_customers_locations = pd.merge(test_customers, test_locations, left_on='akeed_customer_id', right_on='customer_id', how='inner')
print('test_customers_locations shape: ', test_customers_locations.shape)
test_customers_locations.head()

test_customers_locations shape:  (16333, 13)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude
0,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,0,NaN,-9.6e+01,-67.2
1,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,1,NaN,3.9e-02,-78.6
2,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,2,Other,-9.5e+01,43.7
3,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,3,Other,-9.6e+01,43.6
4,ICE2DJP,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,4,Work,2.0e+02,44.1


On créé ici un dataframe ressmeblant a SampleSubmission pour entrainer notre réseau

In [128]:
# keep only the columns we need :CID, LOC_NUM, VENDOR
check_orders = orders_vendors[['customer_id', 'vendor_id']].merge(train_customers_locations[['customer_id', 'location_number']], on='customer_id', how='inner')
check_orders = check_orders[['customer_id','location_number', 'vendor_id']]
check_orders = check_orders.rename(columns={'customer_id': 'CID','location_number': 'LOC_NUM', 'vendor_id': 'VENDOR'})
check_orders = check_orders.drop_duplicates()
print('check_orders shape: ', check_orders.shape)
check_orders.head(10)

check_orders shape:  (181881, 3)


,CID,LOC_NUM,VENDOR
0,92PEE24,0,105
1,92PEE24,1,105
8,I9DNSMJ,0,105
9,I9DNSMJ,1,105
10,I9DNSMJ,2,105
53,I9DNSMJ,0,83
54,I9DNSMJ,1,83
55,I9DNSMJ,2,83
62,I9DNSMJ,0,90
63,I9DNSMJ,1,90


on ajoute une column avec notre prediction, ici la prediction est 1 car on a créé ce df a partir des orders qui ont eu lieux

In [129]:
# add a column predictiion
check_orders['PREDICTION'] = 1
check_orders.head()

,CID,LOC_NUM,VENDOR,PREDICTION
0,92PEE24,0,105,1
1,92PEE24,1,105,1
8,I9DNSMJ,0,105,1
9,I9DNSMJ,1,105,1
10,I9DNSMJ,2,105,1


on a 181881 orders predit, pour entrainer notre reseau on voudrait aussi un nombre de orders du meme ordre de grandeur avec une prediction de 0

In [130]:
# add the same number of rows with PREDICTION = 0
# get the number of rows with PREDICTION = 1
num_rows = check_orders[check_orders['PREDICTION'] == 1].shape[0]

# we need to generate valid pairs of (CID, LOC_NUM, VENDOR) that are not in check_orders
# we can do this by generating all possible pairs and then removing the ones that are in check_orders
# generate all possible pairs, with a maximum of 2 * num_rows
all_pairs = pd.DataFrame(columns=['customer_id', 'location_number', 'vendor_id'])
all_pairs['customer_id'] = np.random.choice(train_customers_locations['customer_id'], 2 * num_rows)
all_pairs['location_number'] = np.random.choice(train_customers_locations['location_number'], 2 * num_rows)
all_pairs['vendor_id'] = np.random.choice(vendors['id'], 2 * num_rows)
all_pairs = all_pairs.drop_duplicates()
print('all_pairs shape: ', all_pairs.shape)

# remove the ones that are in check_orders
all_pairs = all_pairs.merge(check_orders, left_on=['customer_id', 'location_number', 'vendor_id'], right_on=['CID', 'LOC_NUM', 'VENDOR'], how='left')
all_pairs = all_pairs[all_pairs['PREDICTION'].isnull()]

# keep only the columns we need
all_pairs = all_pairs[['customer_id', 'location_number', 'vendor_id']]
all_pairs = all_pairs.rename(columns={'customer_id': 'CID','location_number': 'LOC_NUM', 'vendor_id': 'VENDOR'})
all_pairs['PREDICTION'] = 0
print('all_pairs shape: ', all_pairs.shape)
all_pairs.head()

all_pairs shape:  (352278, 3)
all_pairs shape:  (342734, 4)


,CID,LOC_NUM,VENDOR,PREDICTION
0,KKWEW4N,0,294,0
1,XW90EAP,0,304,0
2,DRR8CO9,0,274,0
4,NW8A814,0,271,0
5,73DGQFO,0,419,0


In [131]:
# merge check_orders and all_pairs
all_pairs = pd.concat([check_orders, all_pairs])
print('all_pairs shape: ', all_pairs.shape)
all_pairs.head()

all_pairs shape:  (524615, 4)


,CID,LOC_NUM,VENDOR,PREDICTION
0,92PEE24,0,105,1
1,92PEE24,1,105,1
8,I9DNSMJ,0,105,1
9,I9DNSMJ,1,105,1
10,I9DNSMJ,2,105,1


avec les infos de CID, LOC_NUM et VENDOR on peut crééer un vecteur d'entrée pour entrainer le reseau sur nos prédictions